In [25]:
import time
import numpy as np
import copy
from aipython.searchGeneric import *
from aipython.searchProblem import *
from aipython.cspProblem import *
from aipython.cspSearch import *


In [26]:
class GameFifteenProblem(Search_problem):
    
    def __init__(self, start, goal):
        self.start = np.array(start)
        self.goal = np.array(goal)

    def start_node(self):
        """Returns the start node"""
        return self.start          
    
    def is_goal(self, node):
        """Returns True if the node is the goal, otherwise False"""
        return np.all(node == self.goal)
    
    def neighbors(self, node):
        """Returns a list of the arcs for the neighbors of node, for example: 
        return [Arc(node, to_neighbor_node1, cost1), Arc(node, to_neighbor_node2, cost2)]"""
        for row in range(4):
            for col in range(4):
                if node[row][col] == 0:
                    zero_x = row
                    zero_y = col
        direction_x = [0,0,-1,1]
        direction_y = [-1,1,0,0]
        nei_list = []
        for i in range(4):
            new_array = copy.deepcopy(node)
            nei_x = zero_x + direction_x[i]
            nei_y = zero_y + direction_y[i]
            if nei_x in range(4) and nei_y in range(4):
                new_array[zero_x][zero_y] = new_array[nei_x][nei_y]
                new_array[nei_x][nei_y] = 0
                nei_list.append(new_array)
        return[Arc(node, nei_list[i], 1) for i in range(len(nei_list))]
    def heuristic(self, node):
        """Returns the heuristic value of the node 
        based on the Manhattan distance"""
        cost = 0
        for i in range(4):
            for j in range(4):
                if node[i][j] == goal[i][j]:
                    continue
                else:
                    target_number = node[i][j]
                    for x in range(4):
                        for y in range(4):
                            if goal[x][y] == target_number:
                                target_x, target_y = x, y
                                cost += (abs(i - target_x) + abs(j - target_y))
        return cost

In [27]:
class BreadthFirstSearcher(Searcher):
    
    def __init__(self,  problem):
        super().__init__(problem)

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return self.frontier.empty()

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        #self.frontier.add(path, self.frontier.frontier_index)
        self.frontier.add(path, path.cost)
    """returns (next) path from the problem's start node
        to a goal node. """
    def search(self):
        visited = set()
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
                #print('visited: ', visited)
                print(len(visited))
                return path
                
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    if (str(arc.to_node) not in visited):
                        visited.add(str(arc.to_node))
                        self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")
            
        
        
                

In [195]:
time1 = time.time()
start = np.array([[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]])

goal = np.array([[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]])


puzzle = GameFifteenProblem(start, goal)
searcher = BreadthFirstSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)
time2 = time.time()
print(time2 - time1)


4347 paths have been expanded and 4463 paths remain in the frontier
8809
Cost:  10
8.380090951919556


In [5]:
# #use a list to check visited nodes
# class BreadthFirstSearcher(Searcher):
    
#     def __init__(self,  problem):
#         super().__init__(problem)
#         self.visited = []

#     """ Initializes the forontier """
#     def initialize_frontier(self):
#         self.frontier = FrontierPQ()

#     """ Returns True if there are no more nodes to expand """
#     def empty_frontier(self):
#         return self.frontier.empty()

#     """ Adds the path to the forontier """
#     def add_to_frontier(self, path):
#         self.frontier.add(path, self.frontier.frontier_index)
#     """returns (next) path from the problem's start node
#         to a goal node. """
#     def search(self):
#         while not self.empty_frontier():
#             path = self.frontier.pop()
#             self.visited.append(path.end())
#             self.display(2, "Expanding:",path,"(cost:",path.cost,")")
#             self.num_expanded += 1
#             if self.problem.is_goal(path.end()):    # solution found
#                 self.display(1, self.num_expanded, "paths have been expanded and",
#                             len(self.frontier), "paths remain in the frontier")
#                 self.solution = path   # store the solution found
#                 return path
#             else:
#                 neighs = self.problem.neighbors(path.end())
#                 self.display(3,"Neighbors are", neighs)
#                 for nb in reversed(list(neighs)):
#                     flag = 0
#                     for ele in self.visited:
#                         if (nb.to_node == ele).all():
#                             flag = 1
# #                     is_in_list = [np.all(nb.to_node) == np.all(ele) for ele in self.visited]
# #                     if True not in is_in_list:
#                     if flag == 0:
#                         self.add_to_frontier(Path(path,nb))
#                 self.display(3,"Frontier:",self.frontier)
#         self.display(1,"No (more) solutions. Total of",
#                      self.num_expanded,"paths expanded.")

In [6]:
# time1 = time.time()
# start = np.array([[1, 2, 3, 4], 
#          [9, 5, 6, 7], 
#          [10, 11, 8, 0], 
#          [13, 14, 15, 12]])

# goal = np.array([[1, 2, 3, 4], 
#         [5, 6, 7, 8], 
#         [9, 10, 11, 12], 
#         [13, 14, 15, 0]])
# start1 = np.array([[1, 2, 3, 4], 
#          [9, 5, 6, 0], 
#          [10, 11, 8, 7], 
#          [13, 14, 15, 12]])
# puzzle = GameFifteenProblem(start, goal)
# searcher = BreadthFirstSearcher(puzzle)
# solution = searcher.search()
# print('Cost: ',  solution.cost)
# print(searcher.frontier.frontier_index)
# time2 = time.time()
# print(time2 - time1)

In [48]:
class IterativeDeepeningSearcher(Searcher):
    def __init__(self, problem):
        super().__init__(problem)

    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = []

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return self.frontier == []

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.append(path)
    
    def search(self):
        depth_tresh = 0
       # visited = set()
        while(True):
            current_depth = 0
            while not self.empty_frontier():
                path = self.frontier.pop()
                self.display(2, "Expanding:",path,"(cost:",path.cost,")")
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):    # solution found
                    self.display(1, self.num_expanded, "paths have been expanded and",
                                len(self.frontier), "paths remain in the frontier")
                    self.solution = path   # store the solution found
                    return path
                else:
                    neighs = self.problem.neighbors(path.end())
                    self.display(3,"Neighbors are", neighs)
                    for arc in reversed(list(neighs)):
                        new_path = Path(path, arc)
                        if new_path.cost <= depth_tresh:
                            #new_path = Path(path,arc)
                            #new_path.cost = arc.cost
                            self.add_to_frontier(new_path)
                    self.display(3,"Frontier:",self.frontier)
            depth_tresh += 1
            #self.frontier.append(start_node)
            self.initialize_frontier()
            self.add_to_frontier(Path(self.problem.start_node()))
            self.num_expanded = 0
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")
            
            

In [49]:
time1 = time.time()
start = np.array([[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]])

goal = np.array([[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]])
puzzle = GameFifteenProblem(start, goal)
searcher = IterativeDeepeningSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)
print(solution)
time2 = time.time()
print('time: ', time2 - time1)

164385 paths have been expanded and 9 paths remain in the frontier
Cost:  10
[[ 2  3  7  4]
 [ 1  6 11  8]
 [ 5 10  0 12]
 [ 9 13 14 15]] --> [[ 2  3  7  4]
 [ 1  6  0  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 2  3  0  4]
 [ 1  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 2  0  3  4]
 [ 1  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 0  2  3  4]
 [ 1  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 1  2  3  4]
 [ 0  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 0 10 11 12]
 [ 9 13 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [ 0 13 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13  0 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14  0 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
time:  9.505352973937988


In [70]:
class IterativeDeepeningAStarSearcher(Searcher):
    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return self.frontier.empty()

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        value = path.cost + self.problem.heuristic(path.end())
        self.frontier.add(path, value)
    
    def search(self):
        h_tresh = 0 + self.problem.heuristic(self.problem.start) #at the beginning, cost = 0, f = 0 + h(start)
        begin = copy.deepcopy(self.frontier)   
        while (True):
            visited = set()
            h_greater = set()
            while not self.empty_frontier():
                path = self.frontier.pop()
                self.display(2, "Expanding:",path,"(cost:",path.cost,")")
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):    # solution found
                    self.display(1, self.num_expanded, "paths have been expanded and",
                                len(self.frontier), "paths remain in the frontier")
                    self.solution = path   # store the solution found
                    print('tresh = ', h_tresh, ', solution found')
                    return path
                else:
                    neighs = self.problem.neighbors(path.end())
                    self.display(3,"Neighbors are", neighs)
                    for arc in reversed(list(neighs)):
                        if str(arc.to_node) not in visited:
                            visited.add(str(arc.to_node))
                            new_path = Path(path, arc)
                            if new_path.cost + self.problem.heuristic(new_path.end()) <= h_tresh:
                                self.add_to_frontier(new_path)
                            else:
                                h_greater.add(new_path.cost + self.problem.heuristic(new_path.end()))
                    self.display(3,"Frontier:",self.frontier)
            self.display(1, "h() tresh = ", h_tresh, "no solution.")
            h_tresh = min(h_greater)
            self.initialize_frontier()
            self.add_to_frontier(Path(self.problem.start_node()))
            self.num_expanded = 0

            

In [72]:
time1 = time.time()
start = [[7, 11, 12, 4],
             [2, 3, 14, 1],
             [6, 5, 13, 0],
             [9, 10, 15, 8]]

goal = [[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]]
puzzle = GameFifteenProblem(start, goal)
searcher = IterativeDeepeningAStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)
print(solution)
time2 = time.time()
print('time: ', time2 - time1)

h() tresh =  32 no solution.
h() tresh =  34 no solution.
h() tresh =  35 no solution.
h() tresh =  36 no solution.
h() tresh =  37 no solution.
h() tresh =  38 no solution.
h() tresh =  39 no solution.
h() tresh =  40 no solution.
h() tresh =  41 no solution.
h() tresh =  42 no solution.
h() tresh =  43 no solution.
h() tresh =  44 no solution.
125514 paths have been expanded and 35247 paths remain in the frontier
tresh =  45 , solution found
Cost:  39
[[ 7 11 12  4]
 [ 2  3 14  1]
 [ 6  5 13  0]
 [ 9 10 15  8]] --> [[ 7 11 12  4]
 [ 2  3 14  1]
 [ 6  5 13  8]
 [ 9 10 15  0]] --> [[ 7 11 12  4]
 [ 2  3 14  1]
 [ 6  5 13  8]
 [ 9 10  0 15]] --> [[ 7 11 12  4]
 [ 2  3 14  1]
 [ 6  5  0  8]
 [ 9 10 13 15]] --> [[ 7 11 12  4]
 [ 2  3  0  1]
 [ 6  5 14  8]
 [ 9 10 13 15]] --> [[ 7 11 12  4]
 [ 2  3  1  0]
 [ 6  5 14  8]
 [ 9 10 13 15]] --> [[ 7 11 12  0]
 [ 2  3  1  4]
 [ 6  5 14  8]
 [ 9 10 13 15]] --> [[ 7 11  0 12]
 [ 2  3  1  4]
 [ 6  5 14  8]
 [ 9 10 13 15]] --> [[ 7  0 11 12]
 [ 2  3

In [10]:
# #v2
# class IterativeDeepeningAStarSearcher(Searcher):
#     """ Initializes the forontier """
#     def initialize_frontier(self):
#         self.frontier = []

#     """ Returns True if there are no more nodes to expand """
#     def empty_frontier(self):
#         return self.frontier == []

#     """ Adds the path to the forontier """
#     def add_to_frontier(self, path):
#         self.frontier.append(path)
    
#     def search(self):
#         h_tresh = self.problem.heuristic(self.problem.start) #at the beginning, cost = 0, f = 0 + h(start)
#         begin = copy.deepcopy(self.frontier) 
#         while (True):
#             h_greater = set()
#             while not self.empty_frontier():
#                 path = self.frontier.pop()
#                 self.display(2, "Expanding:",path,"(cost:",path.cost,")")
#                 self.num_expanded += 1
#                 if self.problem.is_goal(path.end()):    # solution found
#                     self.display(1, self.num_expanded, "paths have been expanded and",
#                                 len(self.frontier), "paths remain in the frontier")
#                     self.solution = path   # store the solution found
#                     print('h_tresh = ', h_tresh, ', solution found')
#                     return path
#                 else:
#                     neighs = self.problem.neighbors(path.end())
#                     self.display(3,"Neighbors are", neighs)
#                     for arc in reversed(list(neighs)):
#                         if  path.cost + 1 + self.problem.heuristic(arc.to_node) <= h_tresh: #path.cost + 1 = current_depth
#                             new_path = Path(path,arc)
#                             self.add_to_frontier(new_path)
#                         else:     
#                             #print("exceed")
#                             h_greater.add(path.cost + 1 + self.problem.heuristic(arc.to_node))
#             self.display(3,"Frontier:",self.frontier)
#             self.display(1, "h() tresh = ", h_tresh, "no solution.")
# #             self.display(1,"No (more) solutions. Total of",
# #                          self.num_expanded,"paths expanded.")
#             h_tresh = min(h_greater)
#             self.frontier = copy.deepcopy(begin)

            

In [11]:
# time1 = time.time()
# start = np.array([[2, 7, 11, 4],
#            [6, 3, 12, 0],
#            [1, 5, 15, 8],
#            [9, 10, 13, 14]])


# goal = np.array([[1, 2, 3, 4], 
#         [5, 6, 7, 8], 
#         [9, 10, 11, 12], 
#         [13, 14, 15, 0]])

# puzzle = GameFifteenProblem(start, goal)
# searcher = IterativeDeepeningAStarSearcher(puzzle)
# solution = searcher.search()
# print('Cost: ',  solution.cost)
# print(solution)
# time2 = time.time()
# print('time: ', time2 - time1)

In [23]:
class UniformCostSearcher(Searcher):
    """ Initializes the forontier """
    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    """ Returns True if there are no more nodes to expand """
    def empty_frontier(self):
        return self.frontier.empty()

    """ Adds the path to the forontier """
    def add_to_frontier(self, path):
        self.frontier.add(path, path.cost)

In [24]:
time1 = time.time()
start = np.array([[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]])


goal = np.array([[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]])

puzzle = GameFifteenProblem(start, goal)
searcher = UniformCostSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)
print(solution)
time2 = time.time()
print('time: ', time2 - time1)


144107 paths have been expanded and 322241 paths remain in the frontier
Cost:  10
[[ 2  3  7  4]
 [ 1  6 11  8]
 [ 5 10  0 12]
 [ 9 13 14 15]] --> [[ 2  3  7  4]
 [ 1  6  0  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 2  3  0  4]
 [ 1  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 2  0  3  4]
 [ 1  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 0  2  3  4]
 [ 1  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 1  2  3  4]
 [ 0  6  7  8]
 [ 5 10 11 12]
 [ 9 13 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 0 10 11 12]
 [ 9 13 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [ 0 13 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13  0 14 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14  0 15]] --> [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
time:  21.706423044204712


In [5]:
grid = [[9, 5, 0, 8, 2, 7, 3, 0, 0],
        [0, 8, 0, 1, 4, 0, 0, 5, 0],
        [0, 1, 0, 5, 9, 0, 0, 0, 0],
        [8, 3, 0, 0, 0, 0, 0, 7, 5],
        [1, 6, 9, 7, 5, 2, 4, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 6, 0],
        [0, 9, 1, 0, 6, 0, 8, 4, 0],
        [7, 0, 8, 0, 3, 1, 0, 0, 6],
        [6, 2, 0, 4, 7, 8, 0, 9, 0]]

In [315]:
# # define constraint function(s)
# def sub_sudoku(matrix):
#     for i in range(1, 10):
#         if i not in matrix:
#             return False
#     return True
# def total_sudoku(grid):
#     grid = np.array(grid)
#     def row_con(grid):
#         for row in range(9):
#             for i in range(1, 10):
#                 if i not in grid[row]:
#                     return False
#         return True
#     def col_con(grid):
#         t_grid = np.array(grid).T
#         return row_con(t_grid)
#     return (row_con(grid) and col_con(grid))
                
                    
# # function that returns a csp
# def grid_to_csp(grid, n):
#     domains = {'M' + str(i): }
#     #constraints = [Constraint(['R'], sudoku(grid, domains))]
#     constraints = [Constraint(['R'+str(i),'R'+str(j)], sudoku(grid,domains)) 
#                    for i in range(n) for j in range(9)]
#     return CSP(domains, constraints)

In [16]:

def sudoku(grid, positioni, positionj):
    #position = [i, j]
    grid = np.array(grid)
    def check(vi, vj): #vi: value of positioni
        if (positioni[0]//3 == positionj[0]//3) and (positioni[1]//3 == positionj[1]//3):
            #means position 1 and position2 are in a same 3*3 matrix
            if vi == vj:
                return False
        #else not in the same matrix
        #if in the same row
        if positioni[0] == positionj[0] and vi == vj:
            return False
        if vi in grid[positioni[0]] or vj in grid[positionj[0]]:
            return False
        #if in the same col
        if positioni[1] == positionj[1] and vi == vj:
            return False
        if vi in grid[:,positioni[1]] or vj in grid[:,positionj[1]]:
            return False
        return True
    return check
             
def grid_to_csp(grid):
    position = []
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                position.append([i, j])
                
    matrix_list = []
    grid = np.array(grid)
    for i in range(0, 7, 3):
        for j in range(0, 7, 3):
            matrix_list.append((grid[i:i+3,j:j+3]))
            
    numbers = list(range(1, 10))
    domains = {}
    for i in range(len(position)):
        possible_num = list(num for num in range(1, 10))
        for row_num in grid[position[i][0]]:
            if row_num in possible_num:
                possible_num.remove(row_num)
        for col_num in grid[:, position[i][1]]:
            if col_num in possible_num:
                possible_num.remove(col_num)
        index = 3 * (position[i][0] // 3) + position[i][1] // 3
        for number in possible_num:
            if number in matrix_list[index]:
                possible_num.remove(number)
        domains['position' + str(position[i])] = possible_num
    #domains = {'position' + str(position[i]): numbers for i in range(len(position))}
    constraints = [Constraint(['position' + str(position[i]), 'position' + str(position[j])], 
                              sudoku(grid, position[i], position[j])) 
                              for i in range(len(position)) for j in range(len(position)) if i != j] 
    return CSP(domains, constraints)

In [17]:

# def row_check(grid, position):
#     def only1(v):
#         return v not in grid[position[0]]
#     return only1
# def col_check(grid, position):
#     def only2(v):
#         return v not in np.array(grid)[:, position[1]]
#     return only2
# def matrix_check(matrix_list, position):
#     index = 3 * (position[0] // 3) + position[1] // 3
#     def only3(v):
#         return v not in matrix_list[index]
#     return only3

    
    
    
# def grid_to_csp(grid):
#     position = []
#     matrix_list = []
#     grid = np.array(grid)
#     for i in range(0, 7, 3):
#         for j in range(0, 7, 3):
#             matrix_list.append((grid[i:i+3,j:j+3]))
#     print(matrix_list)
#     for i in range(9):
#         for j in range(9):
#             if grid[i][j] == 0:
#                 position.append([i, j])
#     domains = {}
#     for i in range(len(position)):
#         possible_num = list(num for num in range(1, 10))
#         for row_num in grid[position[i][0]]:
#             if row_num in possible_num:
#                 possible_num.remove(row_num)
#         for col_num in np.array(grid)[:, position[i][1]]:
#             if col_num in possible_num:
#                 possible_num.remove(col_num)
#         domains['position' + str(position[i])] = possible_num
#     constraints1 = [Constraint(['position' + str(position[i])], 
#                               row_check(grid, position[i])) 
#                               for i in range(len(position))] 
#     constraints2 = [Constraint(['position' + str(position[i])], 
#                               col_check(grid, position[i])) 
#                               for i in range(len(position))] 
#     constraints3 = [Constraint(['position' + str(position[i])], 
#                               matrix_check(matrix_list, position[i])) 
#                               for i in range(len(position))] 
#     constraints = constraints1 + constraints2 + constraints3
#     return CSP(domains, constraints)

In [18]:
# def row_check(grid, position):
#     def only1(v):
#         return v not in grid[position[0]]
#     return only1
# def col_check(grid, position):
#     def only2(v):
#         return v not in np.array(grid)[:, position[1]]
#     return only2
# def matrix_check(matrix_list, position):
#     index = 3 * (position[0] // 3) + position[1] // 3
#     def only3(v):
#         return v not in matrix_list[index]
#     return only3

    
    
    
# def grid_to_csp(grid):
#     position = []
#     matrix_list = []
#     grid = np.array(grid)
#     for i in range(0, 7, 3):
#         for j in range(0, 7, 3):
#             matrix_list.append((grid[i:i+3,j:j+3]))
            
#     for i in range(9):
#         for j in range(9):
#             if grid[i][j] == 0:
#                 position.append([i, j])
#     domains = {}
#     for i in range(len(position)):
#         possible_num = list(num for num in range(1, 10))
#         for row_num in grid[position[i][0]]:
#             if row_num in possible_num:
#                 possible_num.remove(row_num)
#         for col_num in np.array(grid)[:, position[i][1]]:
#             if col_num in possible_num:
#                 possible_num.remove(col_num)
#         domains['position' + str(position[i])] = possible_num
#     constraints = []
#     for i in range(len(position)):
#         constraints.append(Constraint(['position' + str(position[i])], 
#                               row_check(grid, position[i])))
#         constraints.append(Constraint(['position' + str(position[i])], 
#                               col_check(grid, position[i])))
#         constraints.append((Constraint(['position' + str(position[i])], 
#                               matrix_check(matrix_list, position[i]))))
# #     constraints1 = [Constraint(['position' + str(position[i])], 
# #                               row_check(grid, position[i])) , Constraint(['position' + str(position[i])], 
# #                               col_check(grid, position[i])) ,  Constraint(['position' + str(position[i])], 
# #                               matrix_check(matrix_list, position[i]))
# #                               for i in range(len(position))] 
# #     constraints2 = [Constraint(['position' + str(position[i])], 
# #                               col_check(grid, position[i])) 
# #                               for i in range(len(position))] 
# #     constraints3 = [Constraint(['position' + str(position[i])], 
# #                               matrix_check(matrix_list, position[i])) 
# #                               for i in range(len(position))] 
# #     constraints = constraints1 + constraints2 + constraints3
#     return CSP(domains, constraints)

In [19]:
from aipython.cspConsistency import Con_solver
qs = Con_solver(grid_to_csp(grid))
qs.solve_one()


{'position[0, 2]': 6,
 'position[0, 7]': 1,
 'position[0, 8]': 4,
 'position[1, 0]': 2,
 'position[1, 2]': 7,
 'position[1, 5]': 3,
 'position[1, 6]': 6,
 'position[1, 8]': 9,
 'position[2, 0]': 4,
 'position[2, 2]': 3,
 'position[2, 5]': 6,
 'position[2, 6]': 7,
 'position[2, 7]': 8,
 'position[2, 8]': 2,
 'position[3, 2]': 4,
 'position[3, 3]': 6,
 'position[3, 4]': 1,
 'position[3, 5]': 9,
 'position[3, 6]': 2,
 'position[4, 8]': 8,
 'position[5, 0]': 5,
 'position[5, 2]': 2,
 'position[5, 3]': 3,
 'position[5, 5]': 4,
 'position[5, 6]': 9,
 'position[5, 8]': 1,
 'position[6, 0]': 3,
 'position[6, 3]': 2,
 'position[6, 5]': 5,
 'position[6, 8]': 7,
 'position[7, 1]': 4,
 'position[7, 3]': 9,
 'position[7, 6]': 5,
 'position[7, 7]': 2,
 'position[8, 2]': 5,
 'position[8, 6]': 1,
 'position[8, 8]': 3}

In [22]:
# searcher = AStarSearcher(Search_from_CSP(grid_to_csp(grid)))
# print(searcher.search())
time1 = time.time()
searcher = AStarSearcher(Search_from_CSP(grid_to_csp(grid)))
print(searcher.search())
time2 = time.time()
print(time2 - time1)

17684 paths have been expanded and 3 paths remain in the frontier
{} --> {'position[5, 6]': 2} --> {'position[5, 6]': 2, 'position[5, 5]': 9} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3, 'position[2, 2]': 7} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3, 'position[2, 2]': 7, 'position[3, 6]': 9} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3, 'position[2, 2]': 7, 'position[3, 6]': 9, 'position[2, 6]': 6} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3, 'position[2, 2]': 7, 'position[3, 6]': 9, 'position[2, 6]': 6, 'position[6, 8]': 7} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3, 'position[2, 2]': 7, 'position[3, 6]': 9, 'position[2, 6]': 6, 'position[6, 8]': 7, 'position[0, 2]': 6} --> {'position[5, 6]': 2, 'position[5, 5]': 9, 'position[6, 0]': 3, 'position[2, 2]': 7, 'position[3, 6]': 9, 'position[2, 6]': 6, 'pos

In [397]:
grid = [[9, 5, 0, 8, 2, 7, 3, 0, 0],
        [0, 8, 0, 1, 4, 0, 0, 5, 0],
        [0, 1, 0, 5, 9, 0, 0, 0, 0],
        [8, 3, 0, 0, 0, 0, 0, 7, 5],
        [1, 6, 9, 7, 5, 2, 4, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 6, 0],
        [0, 9, 1, 0, 6, 0, 8, 4, 0],
        [7, 0, 8, 0, 3, 1, 0, 0, 6],
        [6, 2, 0, 4, 7, 8, 0, 9, 0]]

In [342]:
matrix_list = []
grid = np.array(grid)
for i in range(0, 7, 3):
    for j in range(0, 7, 3):
        matrix_list.append((grid[i:i+3,j:j+3]))

In [343]:
print(matrix_list)

[array([[9, 5, 0],
       [0, 8, 0],
       [0, 1, 0]]), array([[8, 2, 7],
       [1, 4, 0],
       [5, 9, 0]]), array([[3, 0, 0],
       [0, 5, 0],
       [0, 0, 0]]), array([[8, 3, 0],
       [1, 6, 9],
       [0, 7, 0]]), array([[0, 0, 0],
       [7, 5, 2],
       [0, 8, 0]]), array([[0, 7, 5],
       [4, 3, 0],
       [0, 6, 0]]), array([[0, 9, 1],
       [7, 0, 8],
       [6, 2, 0]]), array([[0, 6, 0],
       [0, 3, 1],
       [4, 7, 8]]), array([[8, 4, 0],
       [0, 0, 6],
       [0, 9, 0]])]


In [344]:
print(matrix_list[0])


[[9 5 0]
 [0 8 0]
 [0 1 0]]


In [347]:
print(5 in matrix_list[0])

True


In [359]:
print(i, i, i) for i in range (4)

SyntaxError: invalid syntax (<ipython-input-359-6265fe338622>, line 1)

In [390]:
def row_check(grid, position):
    def only1(v):
        return v not in grid[position[0]]
    return only1
def col_check(grid, position):
    def only2(v):
        return v not in np.array(grid)[:, position[1]]
    return only2
def matrix_check(matrix_list, position):
    index = 3 * (position[0] // 3) + position[1] // 3
    def only3(v):
        return v not in matrix_list[index]
    return only3
row_check(grid, [0,3])

<function __main__.row_check.<locals>.only1(v)>